<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here
summary = pd.DataFrame({
    'Data Type': df.dtypes,
    'Non-Missing Count': df.count(),
    'Missing Values': df.isnull().sum()
})

print(summary)

                    Data Type  Non-Missing Count  Missing Values
ResponseId              int64              65437               0
MainBranch             object              65437               0
Age                    object              65437               0
Employment             object              65437               0
RemoteWork             object              54806           10631
...                       ...                ...             ...
JobSatPoints_11       float64              29445           35992
SurveyLength           object              56182            9255
SurveyEase             object              56238            9199
ConvertedCompYearly   float64              23435           42002
JobSat                float64              29126           36311

[114 rows x 3 columns]


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here
numeric_stats = df.describe()
print(numeric_stats)

         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000         0.00000   
25%          0.000000 

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here
# Step 1: List unique values (sorted for easier spotting)
unique_countries = sorted(df['Country'].dropna().unique())
print("Unique Country entries:")
print(unique_countries)

# Step 2: Check for missing or empty values
missing_count = df['Country'].isnull().sum()
print(f"\nNumber of missing values in 'Country': {missing_count}")

# Step 3 (optional): Check for entries not in a known list of countries
# Example: Using a predefined list of valid countries
valid_countries = ["United States", "India", "Canada", "United Kingdom", "Germany", "France", "..."]  # add more

invalid_entries = set(df['Country'].dropna().unique()) - set(valid_countries)
print(f"\nPotentially invalid country entries:\n{invalid_entries}")

Unique Country entries:
['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo, Republic of the...', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong (S.A.R.)', 'Hungary

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
## Write your code here
# Define a mapping of inconsistent entries to standardized names
country_mapping = {
    "USA": "United States",
    "U.S.A.": "United States",
    "United States of America": "United States",
    "UK": "United Kingdom",
    "U.K.": "United Kingdom",
    "England": "United Kingdom",
    "Deutschland": "Germany",
    "Germnay": "Germany",
    # Add more mappings as needed
}

# Apply the mapping
df['Country'] = df['Country'].replace(country_mapping)

# Verify changes
print(df['Country'].unique())

['United States' 'United Kingdom of Great Britain and Northern Ireland'
 'Canada' 'Norway' 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria'
 'Switzerland' 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia'
 'Brazil' 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France'
 'Saudi Arabia' 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden'
 'Netherlands' 'Croatia' 'Pakistan' 'Czech Republic'
 'Republic of North Macedonia' 'Finland' 'Slovakia' 'Russian Federation'
 'Greece' 'Israel' 'Belgium' 'Mexico' 'United Republic of Tanzania'
 'Hungary' 'Argentina' 'Portugal' 'Sri Lanka' 'Latvia' 'China' 'Singapore'
 'Lebanon' 'Spain' 'South Africa' 'Lithuania' 'Viet Nam'
 'Dominican Republic' 'Indonesia' 'Kosovo' 'Morocco' 'Taiwan' 'Georgia'
 'San Marino' 'Tunisia' 'Bangladesh' 'Nigeria' 'Liechtenstein' 'Denmark'
 'Ecuador' 'Malaysia' 'Albania' 'Azerbaijan' 'Chile' 'Ghana' 'Peru'
 'Bolivia' 'Egypt' 'Luxembourg' 'Montenegro' 'Cyprus' 'Paraguay'
 'Kazakhstan' 'Slovenia' 'Jordan' 'Venezuela, 

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
## Write your code here
# One-hot encode the 'Employment' column
employment_dummies = pd.get_dummies(df['Employment'], prefix='Employment')

# Concatenate the new one-hot columns back to the original dataframe
df = pd.concat([df, employment_dummies], axis=1)

# Optional: If you want, you can drop the original 'Employment' column
# df.drop('Employment', axis=1, inplace=True)

# Preview the updated dataframe
print(df.head())

   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
## Write your code here
# Count missing values per column
missing_counts = df.isnull().sum()

# Sort descending by missing count
missing_sorted = missing_counts.sort_values(ascending=False)

print("Columns with missing values sorted (highest first):")
print(missing_sorted[missing_sorted > 0])  # show only columns with missing values

Columns with missing values sorted (highest first):
AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
YearsCode                      5568
NEWSOSites                     5151
LearnCode                      4949
EdLevel                        4653
AISelect                       4530
Length: 109, dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [10]:
## Write your code here
mean_value = df['ConvertedCompYearly'].mean()
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(mean_value)

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [11]:
## Write your code here
# Find the most frequent value (mode)
most_freq = df['RemoteWork'].mode()[0]

# Impute missing values with the mode
df['RemoteWork'] = df['RemoteWork'].fillna(most_freq)

### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [12]:
## Write your code here
min_val = df['ConvertedCompYearly'].min()
max_val = df['ConvertedCompYearly'].max()

df['ConvertedCompYearly_Normalized'] = (df['ConvertedCompYearly'] - min_val) / (max_val - min_val)

<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [13]:
## Write your code here
import numpy as np

# Add 1 to avoid log(0) issues if there are zeros
df['ConvertedCompYearly_Log'] = np.log(df['ConvertedCompYearly'] + 1)

### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [15]:
## Write your code here
def convert_yearscodepro(value):
    if pd.isna(value):
        return np.nan
    elif isinstance(value, str):
        if value.lower() == 'less than 1 year':
            return 0.5  # approximate as 0.5 year
        elif value.lower() == 'more than 50 years':
            return 50   # cap at 50
        else:
            try:
                return float(value)
            except:
                return np.nan
    else:
        return value

# Apply conversion
df['YearsCodePro_Num'] = df['YearsCodePro'].apply(convert_yearscodepro)

# Now create ExperienceLevel based on numeric values
def categorize_experience(years):
    if pd.isna(years):
        return 'Unknown'
    elif years < 2:
        return 'Junior'
    elif 2 <= years < 5:
        return 'Mid-level'
    elif 5 <= years < 10:
        return 'Senior'
    else:
        return 'Expert'

df['ExperienceLevel'] = df['YearsCodePro_Num'].apply(categorize_experience)

### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
